<h1>YouTube Playlist Downloader</h1>
<p>This script allows you to download videos from a YouTube playlist, handling age-restricted videos by logging
    in with your credentials.</p>

<h2>Explanation of the Script</h2>
<p>The script uses Python and the <code>yt-dlp</code> library to download videos. Here is a breakdown of the
    components:</p>
<ul>
    <li><strong>Importing necessary modules:</strong> Imports required libraries such as <code>os</code> for
        path operations, <code>ThreadPoolExecutor</code> for concurrent downloading, <code>tqdm</code> for
        progress bars, <code>yt_dlp</code> for downloading videos, and <code>sys</code> for handling
        command-line arguments.</li>
    <li><strong>download_video function:</strong> Downloads a single video. The <code>outtmpl</code> is set
        dynamically for each video to ensure unique filenames based on their titles and indices.</li>
    <li><strong>get_name function:</strong> Generates a unique name for each video based on its title and index
        within the playlist.</li>
    <li><strong>download_playlist function:</strong>
        <ul>
            <li>Creates the download directory if it does not exist.</li>
            <li>Reads YouTube credentials from a file named <code>.cred</code>.</li>
            <li>Extracts video information from the playlist URL.</li>
            <li>Prepares a list of video URLs with their corresponding names.</li>
            <li>Downloads videos concurrently using <code>ThreadPoolExecutor</code>.</li>
        </ul>
    </li>
    <li><strong>Main block:</strong> Takes the playlist URL from the command-line arguments and calls the
        <code>download_playlist</code> function.
    </li>
</ul>

<h2>Script</h2>

```Python
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import yt_dlp as youtube_dl
import sys
# Function to download a single video using yt-dlp
def download_video(video_info, download_path, ydl_opts):
    try:
        ydl_opts['outtmpl'] = os.path.join(download_path, video_info['name'])
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_info['url']])
    except Exception as e:
        print(f"Error downloading {video_info['name']}: {e}")

# Function to generate unique names for videos
def get_name(ep, title, ext):
    return f"{title.strip()}{ep}.{ext.strip()}"

# Main function to download the playlist
def download_playlist(playlist_url, download_path='downloads', max_workers=8):
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    ydl_opts = {
        'format': 'best',
        'username': None,
        'password': None,
        'verbose': False,
    }

    # Read credentials from file
    try:
        with open('.cred', 'r') as file:
            lines = file.readlines()
            ydl_opts['username'] = lines[0].strip().split(':')[1]
            ydl_opts['password'] = lines[1].strip().split(':')[1]
    except FileNotFoundError:
        print("Credentials file not found. Make sure '.cred' exists and is formatted correctly.")
        return

    # Extract video URLs from the playlist
    try:
        ydl = youtube_dl.YoutubeDL(ydl_opts)
        result = ydl.extract_info(playlist_url, download=False)
        video_urls = [{'name': get_name(i + 1, e['title'], e['video_ext']), 'url': e['url']}
                      for i, e in enumerate(result['entries'])]
    except Exception as e:
        print(f"Error extracting playlist information: {e}")
        return

    # Download videos concurrently
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(lambda url: download_video(url, download_path, ydl_opts), video_urls), total=len(video_urls)))

# Example usage
if __name__ == "__main__":
    playlist_url = sys.argv[1]
    download_playlist(playlist_url)
```

<h2>Usage</h2>
<ol>
    <li>Ensure your <code>.cred</code> file is correctly formatted and located in the same directory as your
        script:
        <pre>
username:your_email@example.com
password:your_password
            </pre>
    </li>
    <li>Run the script from the command line with the playlist URL as an argument:
        <pre>
python script_name.py YOUR_PLAYLIST_URL
            </pre>
    </li>
</ol>

<p>This script should now download each video from the playlist with unique filenames, handling both the naming
    and downloading process efficiently.</p>

In [1]:
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import yt_dlp as youtube_dl
import sys
# Function to download a single video using yt-dlp


def download_video(video_info, download_path, ydl_opts):
    try:
        ydl_opts['outtmpl'] = os.path.join(download_path, video_info['name'])
        if os.path.exists(ydl_opts['outtmpl']):
            return
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_info['url']])
    except Exception as e:
        print(f"Error downloading {video_info['name']}: {e}")

# Main function to download playlist


def get_name(ep, title, ext):
    return f"{title.strip()}{ep}.{ext.strip()}"


def download_playlist(playlist_url, download_path='downloads', max_workers=8):
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    ydl_opts = {
        'format': 'best',
        'username': None,
        'password': None,
        'verbose': False,
    }

    # Read credentials from file
    try:
        with open('.cred', 'r') as file:
            lines = file.readlines()
            ydl_opts['username'] = lines[0].strip().split(':')[1]
            ydl_opts['password'] = lines[1].strip().split(':')[1]
    except FileNotFoundError:
        print("Credentials file not found. Make sure '.cred' exists and is formatted correctly.")
        return

    # Extract video URLs from the playlist
    try:
        ydl = youtube_dl.YoutubeDL(ydl_opts)
        result = ydl.extract_info(playlist_url, download=False)
        video_urls = [{'name': get_name(i+1, e['title'], e['video_ext']), 'url': e['url']}
                      for i, e in enumerate(result['entries'])]
    except Exception as e:
        print(f"Error extracting playlist information: {e}")
        return

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(lambda url: download_video(
            url, download_path, ydl_opts), video_urls), total=len(video_urls)))

In [10]:
url = 'https://www.youtube.com/playlist?list=PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey'
ydl_opts = {
    'format': 'best',
    'username': None,
    'password': None,
    'verbose': False,
}

# Read credentials from file
try:
    with open('.cred', 'r') as file:
        lines = file.readlines()
        ydl_opts['username'] = lines[0].strip().split(':')[1]
        ydl_opts['password'] = lines[1].strip().split(':')[1]
except FileNotFoundError:
    print("Credentials file not found. Make sure '.cred' exists and is formatted correctly.")
ydl = youtube_dl.YoutubeDL(ydl_opts)

result = ydl.extract_info(url, download=False)

[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey
[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey: Downloading webpage
[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: The Queen of SOP 2胜女的代价
[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey page 1: Downloading API JSON


[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey page 1: Downloading API JSON


[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey page 1: Downloading API JSON


[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey page 1: Downloading API JSON


[youtube:tab] Playlist The Queen of SOP 2胜女的代价: Downloading 34 items of 34
[download] Downloading item 1 of 34


[youtube] Extracting URL: https://www.youtube.com/watch?v=Qs8x-THpwf0
[youtube] Qs8x-THpwf0: Downloading webpage
[youtube] Qs8x-THpwf0: Downloading ios player API JSON
[youtube] Qs8x-THpwf0: Downloading m3u8 information
[download] Downloading item 2 of 34
[youtube] Extracting URL: https://www.youtube.com/watch?v=YyspDj2Y4uM
[youtube] YyspDj2Y4uM: Downloading webpage
[youtube] YyspDj2Y4uM: Downloading ios player API JSON
[youtube] YyspDj2Y4uM: Downloading m3u8 information
[download] Downloading item 3 of 34
[youtube] Extracting URL: https://www.youtube.com/watch?v=oYwjuGlF2n8
[youtube] oYwjuGlF2n8: Downloading webpage
[youtube] oYwjuGlF2n8: Downloading ios player API JSON
[youtube] oYwjuGlF2n8: Downloading m3u8 information
[download] Downloading item 4 of 34
[youtube] Extracting URL: https://www.youtube.com/watch?v=MedBnTzNqqA
[youtube] MedBnTzNqqA: Downloading webpage
[youtube] MedBnTzNqqA: Downloading ios player API JSON
[youtube] MedBnTzNqqA: Downloading m3u8 information
[download] D

In [21]:
[(i+1, r['url'], r['webpage_url']) for i, r in enumerate(result['entries'])]

[(1,
  'https://rr3---sn-cu-ac5l.googlevideo.com/videoplayback?expire=1717274698&ei=6jNbZu-hMb2xp-oPncS2wAY&ip=150.143.112.161&id=o-AAEJo7ECod6xQwOKB1Pex3hgBEyPHvHAinPbay4jU2EL&itag=22&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&mh=gI&mm=31%2C29&mn=sn-cu-ac5l%2Csn-cu-auod&ms=au%2Crdu&mv=m&mvi=3&pl=25&initcwndbps=2303750&bui=AbKP-1PZRSTeHq9itLzdfqBXrpHyzGe_Ok2dAz_6xtmx8BNor8b1zcVVMpbSSorozeDWCI190Jojp_vc&spc=UWF9f2GhzUJ5cT0S_2m01nQV1sfoMM0gEObX3P57ILWhC4kalTIIT9kZVsyj&vprv=1&svpuc=1&mime=video%2Fmp4&ns=xRpjmhoEjwjwLcl3Ix9bYScQ&rqh=1&cnr=14&ratebypass=yes&dur=2760.109&lmt=1715507443084848&mt=1717252608&fvip=4&c=WEB&sefc=1&txp=5318224&n=daKfupnc89NG6A&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Ccnr%2Cratebypass%2Cdur%2Clmt&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AHlkHjAwRQIgSZCIvWH9bNrIm96TMvmYgPsa9jT5lC211OoVoEdeRZoCIQDXkOjysn_z1AJpVOcExJCoOXof0vSnBsirzi-NwQxi7Q%3D%3D&sig=AJfQdSswRAIgYRiCX10z

In [25]:
ydl._outtmpl_expandpath('downloadss')

'downloadss'

In [31]:
ydl.extract_info(url)

[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey
[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey: Downloading webpage
[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: The Queen of SOP 2胜女的代价
[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey page 1: Downloading API JSON


[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey page 1: Downloading API JSON


[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey page 1: Downloading API JSON


[youtube:tab] PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey page 1: Downloading API JSON


[youtube:tab] Playlist The Queen of SOP 2胜女的代价: Downloading 34 items of 34
[download] Downloading item 1 of 34
[youtube] Extracting URL: https://www.youtube.com/watch?v=Qs8x-THpwf0
[youtube] Qs8x-THpwf0: Downloading webpage
[youtube] Qs8x-THpwf0: Downloading ios player API JSON
[youtube] Qs8x-THpwf0: Downloading m3u8 information
[info] Qs8x-THpwf0: Downloading 1 format(s): 22
[download] The Queen of SOP 2 胜女的代价 第1集【主演：郑爽、张翰】 [Qs8x-THpwf0].mp4 has already been downloaded
[download] 100% of  383.06MiB
[download] Downloading item 2 of 34
[youtube] Extracting URL: https://www.youtube.com/watch?v=YyspDj2Y4uM
[youtube] YyspDj2Y4uM: Downloading webpage
[youtube] YyspDj2Y4uM: Downloading ios player API JSON
[youtube] YyspDj2Y4uM: Downloading m3u8 information
[info] YyspDj2Y4uM: Downloading 1 format(s): 22
[download] The Queen of SOP 2胜女的代价 第2集【主演：郑爽、张翰】 [YyspDj2Y4uM].mp4 has already been downloaded
[download] 100% of  361.04MiB
[download] Downloading item 3 of 34
[youtube] Extracting URL: http

{'id': 'PLh3Gw78xy9ErnUJMIie-isooci1MJr1Ey',
 'title': 'The Queen of SOP 2胜女的代价',
 'availability': 'public',
 'channel_follower_count': None,
 'description': '',
 'tags': [],
 'thumbnails': [{'url': 'https://i.ytimg.com/vi/Qs8x-THpwf0/hqdefault.jpg?sqp=-oaymwEWCKgBEF5IWvKriqkDCQgBFQAAiEIYAQ==&rs=AOn4CLDj9r_OizxjAhA1QSw8jR9cYA3luw',
   'height': 94,
   'width': 168,
   'id': '0',
   'resolution': '168x94'},
  {'url': 'https://i.ytimg.com/vi/Qs8x-THpwf0/hqdefault.jpg?sqp=-oaymwEWCMQBEG5IWvKriqkDCQgBFQAAiEIYAQ==&rs=AOn4CLAA9Du4LwcOzca6NENJq6xc98CLcw',
   'height': 110,
   'width': 196,
   'id': '1',
   'resolution': '196x110'},
  {'url': 'https://i.ytimg.com/vi/Qs8x-THpwf0/hqdefault.jpg?sqp=-oaymwEXCPYBEIoBSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBv5qD9ygdpnYoTRGVtAbokzNxQqA',
   'height': 138,
   'width': 246,
   'id': '2',
   'resolution': '246x138'},
  {'url': 'https://i.ytimg.com/vi/Qs8x-THpwf0/hqdefault.jpg?sqp=-oaymwEXCNACELwBSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLApTPsqmnmSL-2602G6rDUJ0Y0QfQ',